<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries
This section is somewhat intimidating to new users, but modifications are necessary only to access advanced features.

In [3]:
!git clone --depth 1 https://github.com/profteachkids/chetools.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/chetools") #Path to CHE module imports

In [4]:
from dotmap import DotMap
import pandas as pd
import jax
import jax.numpy as jnp
from jax.config import config
from jax.experimental.host_callback import id_print
config.update("jax_enable_x64", True) #JAX default is 32bit single precision

from tools.tree_array_transform2 import VSC, Comp, Range
import tools.che as che

## Dot-access to chemical properties and calculations
ChemCAD data properties and binary interaction parameters can be exported as text files.  A few common chemicals are stored in a Github repository.  Users with ChemCad access can export and access their own data files.  The *che.Props* class parses these files for convenient dot-access and implementations of common calculations.

In [5]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [6]:
print(f'Tc: {p.Tc}')
print(f'Pc: {p.Tc}')
print(f'Vapor Pressure at 432.1 K {p.Pvap(432.1)} Pa')
print(f'NRTL activity coefficients for equimolar mixture {p.NRTL_gamma([1/3,1/3,1/3], 300)} ')

Tc: [513.92 508.31 647.35]
Pc: [513.92 508.31 647.35]
Vapor Pressure at 432.1 K [1220670.06610508 1069634.05615236  600793.6517417 ] Pa
NRTL activity coefficients for equimolar mixture [1.11669551 1.12519412 1.85757473] 


## Adiabatic Flash Calculation
Model mass/energy balance and VLE - all variables (knowns and unknowns) are in a convenient DotMap structure (c).  The model is written once.

Sum up weighted square deviations for minimization.
 

In [7]:
def model(c):
    # c: combination of adjustable variables and static state parameters
    # r: DotMap - store intermediate results for reporting
    r=DotMap()
    r.V = c.Vy * c.Vtot # Moles of each component = mole fractions * total moles
    r.L = c.Lx * c.Ltot
    r.F = c.Fz * c.Ftot
    mass_balance = r.F - r.V - r.L # Mass balance for each component (vectors!)

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    r.FH = p.Hl(nL=r.F, T=c.FT)
    r.VH = p.Hv(nV=r.V, T=c.flashT)
    r.LH = p.Hl(nL=r.L, T=c.flashT)
    energy_balance = (r.FH - r.VH - r.LH)

    # Raoults with NRTL activity coefficient correction.  One-liner!
    r.fugL = c.Lx  * p.NRTL_gamma(c.Lx,c.flashT)* p.Pvap(c.flashT)
    r.fugV = c.Vy*c.flashP
    VLE = r.fugL - r.fugV
    id_print([mass_balance, energy_balance, VLE])
    return [mass_balance, energy_balance, VLE], r

## What's known (static) and unknown (variable) - automatically combined and tracked.

What's known and unknown can be easily swapped around without having to modify the model.  The DotMap structures can have nested lists of DotMaps with nested lists of arrays, and so forth.

In [8]:
# Static parameters (Total feed, feed mole fractions, feed temperature and )
c=DotMap()
c.Ftot=10 # Total Feed moles
c.Fz = jnp.array([1/3, 1/3, 1/3]) # Equimolar feed composition
c.FT = 450 # Feed temperature
c.flashP= 101325 # Flash drum pressure

c.Vy = Comp(c.Fz) # Guess vapor/liquid composition equal to feed
c.Lx = Comp(c.Fz) # Comp - constrains mole fractions to behave like mole fractions!
c.flashT = Range(360, 273.15, c.FT)  # Guess and bounds for flash temperature
c.Vtot = Range(c.Ftot/2, 0., c.Ftot)  # Guess half of feed in vapor
c.Ltot = Range(c.Ftot/2, 0., c.Ftot)

## Magic happens here
The VSC class combines unknown variables and static parameters into a convenient DotMap structure.  Transformations between known/unknown DotMaps and the flat arrays required by the minimization routine is automated.

The minimization algorithm is a robust Trust-Newton Conjugate Gradient coded to take advantage of JAX automatic Jacobian vector products.  

The initial JAX model compilation is a bit slow, but subsequent repeat calculations are fast.

In [9]:
vsc=VSC(c, model)
vsc.solve(jit=True, verbosity=0)

[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [17774.2642706  11514.40996588  3103.78604499] ]
[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [17774.2642706  11514.40996588  3103.78604499] ]
[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [17774.2642706  11514.40996588  3103.78604499] ]
[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [17774.2642706  11514.40996588  3103.78604499] ]
[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [17774.2642706  11514.40996588  3103.78604499] ]
[ [ 0.05170464  0.02811635 -0.07982099]
  -509.25522526
  [1794.96583178 1067.1330319   247.94350251] ]
[ [ 0.05170464  0.02811635 -0.07982099]
  -509.25522526
  [1794.96583178 1067.1330319   247.94350251] ]
[ [-2.09536091e-05  4.40752856e-04 -4.19799247e-04]
  -10.69974963
  [ 5.76406407 20.28184402 11.89291816] ]
[ [-2.09536091e-05  4.40752856e-04 -4.19799247e-04]
  -10.69974963
  [ 5.76406407 20.28184402

In [10]:
# State parameters
vsc.sdf

vector1   vector3                    
             1         1         2         3
Ftot        10                              
flashP  101325                              
Fz              0.333333  0.333333  0.333333
FT         450

In [11]:
# Adjustable Variables
vsc.vdf

vector1   vector3                    
              1         1         2         3
Ltot     6.0261                              
Lx               0.311418  0.326803  0.361779
Vtot     3.9739                              
Vy               0.366567  0.343236  0.290198
flashT  352.966

In [12]:
# Intermediate results
vsc.rdf



vector3                        vector1
            1        2        3            1
V      1.4567  1.36398  1.15322             
L     1.87664  1.96935  2.18012             
F     3.33333  3.33333  3.33333             
FH                              -2.70752e+06
VH                                   -979739
LH                              -1.72779e+06
fugL  37142.4  34778.4  29404.3             
fugV  37142.4  34778.4  29404.3

In [13]:
vsc.cdf


vector1   vector3                    
              1         1         2         3
FT          450                              
Ftot         10                              
Fz               0.333333  0.333333  0.333333
Ltot     6.0261                              
Lx               0.311418  0.326803  0.361779
Vtot     3.9739                              
Vy               0.366567  0.343236  0.290198
flashP   101325                              
flashT  352.966

In [14]:
vsc.r.FH - vsc.r.VH - vsc.r.LH

DeviceArray(4.65661287e-10, dtype=float64)

In [15]:
vsc.r.fugL - vsc.r.fugV

DeviceArray([3.49245965e-10, 7.27595761e-12, 1.16415322e-10], dtype=float64)